# Análise de Dados de Ocorrências Aeronáuticas - CENIPA

## Desafio de Ciência de Dados Di2win

## Notebook 01 - Processo de extração, transformação e carregamento dos dados (ETL)

Este notebook documenta a fase de ETL dos dados.

### 0. Instalação das Bibliotecas

Esta célula inicial garante que todas as bibliotecas necessárias para a análise estejam instaladas no ambiente.

In [ ]:
# O '-q' é para uma instalação "quieta" (quiet), sem exibir muitas mensagens
!pip install -q pandas matplotlib seaborn


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 1. Inspeção Inicial dos Dados

O primeiro passo fundamental é carregar cada um dos arquivos `.csv` em um DataFrame do Pandas e realizar uma inspeção básica para entender sua estrutura. Para cada arquivo, vamos verificar:

1.  `.head()`: Para visualizar as 5 primeiras linhas e ter uma noção do conteúdo.
2.  `.info()`: Para entender os tipos de dados de cada coluna e a contagem de valores não-nulos.
3.  `.shape`: Para saber o número exato de linhas e colunas.

In [1]:
import pandas as pd

# --- OCORRÊNCIA ---
print("="*40)
print("INSPECIONANDO O ARQUIVO: ocorrencia.csv")
print("="*40)
df_ocorrencia = pd.read_csv('data/ocorrencia.csv', sep=';', encoding='latin1', low_memory=False)
print("\nFormato do DataFrame (linhas, colunas):", df_ocorrencia.shape)
print("\nInformações das Colunas e Tipos de Dados:")
df_ocorrencia.info()
print("\nVisualizando as 5 primeiras linhas:")
display(df_ocorrencia.head())

INSPECIONANDO O ARQUIVO: ocorrencia.csv

Formato do DataFrame (linhas, colunas): (13185, 22)

Informações das Colunas e Tipos de Dados:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13185 entries, 0 to 13184
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   codigo_ocorrencia               13185 non-null  int64 
 1   codigo_ocorrencia1              13185 non-null  int64 
 2   codigo_ocorrencia2              13185 non-null  int64 
 3   codigo_ocorrencia3              13185 non-null  int64 
 4   codigo_ocorrencia4              13185 non-null  int64 
 5   ocorrencia_classificacao        13185 non-null  object
 6   ocorrencia_latitude             10425 non-null  object
 7   ocorrencia_longitude            10425 non-null  object
 8   ocorrencia_cidade               13185 non-null  object
 9   ocorrencia_uf                   13185 non-null  object
 10  ocorrencia_pais               

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
0,87125,87125,87125,87125,87125,INCIDENTE,-7.219166666666,-39.26944444444,JUAZEIRO DO NORTE,CE,...,11/05/2025,04:20:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
1,87124,87124,87124,87124,87124,INCIDENTE,-18.88361111111,-48.22527777777,UBERLÂNDIA,MG,...,08/05/2025,14:00:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
2,87123,87123,87123,87123,87123,INCIDENTE,-23.43555555555,-46.47305555555,GUARULHOS,SP,...,09/05/2025,18:45:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
3,87122,87122,87122,87122,87122,INCIDENTE,-29.71083333333,-53.69222222222,SANTA MARIA,RS,...,04/05/2025,14:45:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
4,87121,87121,87121,87121,87121,INCIDENTE,-20.81722222222,-49.40694444444,SÃO JOSÉ DO RIO PRETO,SP,...,10/05/2025,10:00:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO


### 1.1. Primeiras Observações sobre `ocorrencia.csv`

Com base na inspeção inicial, levantamos os seguintes pontos cruciais sobre este arquivo:

* **Chaves e Colunas Repetidas:** Existem 5 colunas de código (`codigo_ocorrencia` a `codigo_ocorrencia4`). Nossa hipótese é que elas servem como "chaves estrangeiras" para conectar este `DataFrame` com os outros (aeronave, recomendação, etc.). Precisamos verificar se os valores nelas são idênticos.
* **Tipos de Dados:** Colunas de data e hora (`ocorrencia_dia`, `ocorrencia_hora`) estão como texto (`object`). Para qualquer análise temporal, será necessário convertê-las para o tipo `datetime`.
* **Dados Ausentes (Nulos):** Há uma quantidade significativa de valores nulos, especialmente em:
    * `ocorrencia_latitude` / `ocorrencia_longitude`
    * `divulgacao_relatorio_numero`
    * `divulgacao_dia_publicacao` (a grande maioria é nula)
* **Valores Sentinela:** Uma inspeção visual no `.head()` revela o uso de `***` em `divulgacao_relatorio_numero`. Este é um "valor sentinela" usado para representar dados não aplicáveis ou não preenchidos. Precisamos tratar isso como um valor nulo padrão (`NaN`) para que o Pandas possa lidar com ele corretamente.

Nossa próxima etapa será focar em validar essas hipóteses e iniciar a limpeza e o pré-processamento dos dados.

In [2]:
print("--- Investigando e Limpando as Colunas de Código em 'df_ocorrencia' ---")

# 1. Verificando se os valores nas 5 colunas de código são idênticos para cada linha
colunas_codigo = ['codigo_ocorrencia', 'codigo_ocorrencia1', 'codigo_ocorrencia2', 'codigo_ocorrencia3', 'codigo_ocorrencia4']
sao_identicas = (df_ocorrencia[colunas_codigo].nunique(axis=1) == 1).all()


# 2. Agindo com base na verificação
if sao_identicas:
    print("\n[CONFIRMADO] A hipótese é verdadeira. Os valores nas 5 colunas de código são idênticos para todas as linhas.")
    
    # Se são idênticas, podemos remover as 4 colunas redundantes com segurança
    colunas_para_remover = ['codigo_ocorrencia1', 'codigo_ocorrencia2', 'codigo_ocorrencia3', 'codigo_ocorrencia4']
    df_ocorrencia_limpo = df_ocorrencia.drop(columns=colunas_para_remover)
    
    print(f"\nColunas removidas: {colunas_para_remover}")
    print("\nO DataFrame 'df_ocorrencia' agora possui apenas a chave 'codigo_ocorrencia'.")
    
else:
    print("\n[FALHA] A hipótese é falsa. Existem linhas onde os valores das colunas de código são diferentes.")
    # Se não forem idênticas, não fazemos nada por enquanto e mantemos o dataframe original
    df_ocorrencia_limpo = df_ocorrencia.copy()


# --- Resultados ---
print("\nVisualizando a estrutura do novo DataFrame 'df_ocorrencia_limpo':")
display(df_ocorrencia_limpo.head())
print(f"\nNovo formato do DataFrame (linhas, colunas): {df_ocorrencia_limpo.shape}")

--- Investigando e Limpando as Colunas de Código em 'df_ocorrencia' ---

[CONFIRMADO] A hipótese é verdadeira. Os valores nas 5 colunas de código são idênticos para todas as linhas.

Colunas removidas: ['codigo_ocorrencia1', 'codigo_ocorrencia2', 'codigo_ocorrencia3', 'codigo_ocorrencia4']

O DataFrame 'df_ocorrencia' agora possui apenas a chave 'codigo_ocorrencia'.

Visualizando a estrutura do novo DataFrame 'df_ocorrencia_limpo':


,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,ocorrencia_pais,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
0,87125,INCIDENTE,-7.219166666666,-39.26944444444,JUAZEIRO DO NORTE,CE,BRASIL,FAER,11/05/2025,04:20:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
1,87124,INCIDENTE,-18.88361111111,-48.22527777777,UBERLÂNDIA,MG,BRASIL,SBUL,08/05/2025,14:00:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
2,87123,INCIDENTE,-23.43555555555,-46.47305555555,GUARULHOS,SP,BRASIL,SBGR,09/05/2025,18:45:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
3,87122,INCIDENTE,-29.71083333333,-53.69222222222,SANTA MARIA,RS,BRASIL,SBSM,04/05/2025,14:45:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
4,87121,INCIDENTE,-20.81722222222,-49.40694444444,SÃO JOSÉ DO RIO PRETO,SP,BRASIL,SBSR,10/05/2025,10:00:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO



Novo formato do DataFrame (linhas, colunas): (13185, 18)


### 1.2. Limpeza de Tipos de Dados e Tratamento de Valores Sentinela

Com a chave primária corrigida, o próximo passo da limpeza é tratar duas outras questões importantes que identificamos no `df_ocorrencia_limpo`:

1.  **Conversão de Data e Hora:** As colunas `ocorrencia_dia` e `ocorrencia_hora` estão como texto (`object`). Vamos convertê-las para o tipo `datetime`, que é o formato correto para manipulação de datas e horas, e uni-las em uma única coluna `ocorrencia_data_hora` para facilitar análises temporais:
    > a) **Pré-tratamento da Data:** Antes de qualquer tentativa de conversão, vamos aplicar um `.str.strip()` à coluna `ocorrencia_dia` para remover esses caracteres indesejados.
    
    > b) **Conversão Padrão:** Após a limpeza, tentaremos a conversão vetorizada padrão (`pd.to_datetime`).
2.  **Substituição de Valores Sentinela:** Vamos percorrer o `DataFrame` para substituir todos os marcadores de texto como `***` e outros formatos não-padrão por um valor nulo (`NaN`), que o Pandas entende como "ausente". Isso nos dará uma visão precisa da quantidade real de dados faltantes.

In [ ]:
print("--- Limpando Tipos de Dados e Valores Sentinela em 'df_ocorrencia_limpo' ---")

# Vamos usar o df_ocorrencia_limpo que já não tem as colunas de código duplicadas
df_ocorrencia_tratado = df_ocorrencia_limpo.copy()

# 1. Pré-tratamento da coluna 'ocorrencia_dia' para remover espaços/caracteres ocultos
# Primeiro, garantimos que a coluna é do tipo string para usar o .str
df_ocorrencia_tratado['ocorrencia_dia'] = df_ocorrencia_tratado['ocorrencia_dia'].astype(str)
df_ocorrencia_tratado['ocorrencia_dia'] = df_ocorrencia_tratado['ocorrencia_dia'].str.strip()


# 2. Tratamento de data e hora (AGORA com a coluna pré-tratada)
df_ocorrencia_tratado['ocorrencia_dia'] = pd.to_datetime(df_ocorrencia_tratado['ocorrencia_dia'], format='%d/%m/%Y', errors='coerce')
df_ocorrencia_tratado['ocorrencia_hora'] = pd.to_timedelta(df_ocorrencia_tratado['ocorrencia_hora'], errors='coerce')
df_ocorrencia_tratado['ocorrencia_data_hora'] = df_ocorrencia_tratado['ocorrencia_dia'] + df_ocorrencia_tratado['ocorrencia_hora']

print("Etapa de conversão de data e hora concluída.")


# 3. Substituindo valores sentinela e tratando inconsistências
for col in df_ocorrencia_tratado.select_dtypes(include=['object']).columns:
    df_ocorrencia_tratado[col] = df_ocorrencia_tratado[col].str.strip()
    df_ocorrencia_tratado[col] = df_ocorrencia_tratado[col].replace(['***', '****', '*****', 'NULL', 'NÃO IDENTIFICADA'], pd.NA, regex=False)

print("Valores sentinela ('***', etc.) substituídos por valores nulos.")


# 4. Verificando a situação final dos tipos e dos nulos
print("\nInformações do DataFrame após a limpeza revisada:")
df_ocorrencia_tratado.info()

--- Limpando Tipos de Dados e Valores Sentinela em 'df_ocorrencia_limpo' (Abordagem Revisada) ---
Etapa de conversão de data e hora concluída.
Valores sentinela ('***', etc.) substituídos por valores nulos.

Informações do DataFrame após a limpeza revisada:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13185 entries, 0 to 13184
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype          
---  ------                          --------------  -----          
 0   codigo_ocorrencia               13185 non-null  int64          
 1   ocorrencia_classificacao        13185 non-null  object         
 2   ocorrencia_latitude             9161 non-null   object         
 3   ocorrencia_longitude            9160 non-null   object         
 4   ocorrencia_cidade               13127 non-null  object         
 5   ocorrencia_uf                   13164 non-null  object         
 6   ocorrencia_pais                 13185 non-null  object         
 7   ocorren

In [4]:
df_ocorrencia_limpo.head()

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,ocorrencia_pais,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista,ocorrencia_data_hora
0,87125,INCIDENTE,-7.219166666666,-39.26944444444,JUAZEIRO DO NORTE,CE,BRASIL,FAER,2025-11-05,0 days 04:20:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO,2025-11-05 04:20:00
1,87124,INCIDENTE,-18.88361111111,-48.22527777777,UBERLÂNDIA,MG,BRASIL,SBUL,2025-08-05,0 days 14:00:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO,2025-08-05 14:00:00
2,87123,INCIDENTE,-23.43555555555,-46.47305555555,GUARULHOS,SP,BRASIL,SBGR,2025-09-05,0 days 18:45:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO,2025-09-05 18:45:00
3,87122,INCIDENTE,-29.71083333333,-53.69222222222,SANTA MARIA,RS,BRASIL,SBSM,2025-04-05,0 days 14:45:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO,2025-04-05 14:45:00
4,87121,INCIDENTE,-20.81722222222,-49.40694444444,SÃO JOSÉ DO RIO PRETO,SP,BRASIL,SBSR,2025-10-05,0 days 10:00:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO,2025-10-05 10:00:00


### 1.3. Análise e Estratégia para Dados Ausentes

A limpeza de tipos e de valores sentinela nos revelou a real quantidade de dados faltantes em cada coluna. Agora, precisamos de uma estratégia para lidar com eles.

Nossa análise da contagem de nulos mostra os seguintes pontos críticos:

* **Localização (`ocorrencia_latitude`, `ocorrencia_longitude`):** Possuem cerca de 4.000 valores nulos. É inviável "inventar" ou preencher esses dados.
    * **Estratégia:** Para análises que dependem de mapas ou geolocalização, teremos que trabalhar com o subconjunto de dados que possui essa informação. Para outras análises, podemos ignorar essas colunas.

* **Data da Ocorrência (`ocorrencia_dia`, `ocorrencia_data_hora`):** A conversão para `datetime` resultou em mais de 7.800 valores nulos. Isso é um **ponto de atenção gravíssimo**. Significa que mais da metade das datas no formato original não puderam ser convertidas. Precisaremos investigar o formato dessas datas inválidas.
    * **Estratégia Imediata:** Qualquer análise temporal ficará restrita aos ~5.300 registros com data válida.

* **Dados da Divulgação (`divulgacao_relatorio_numero`, `divulgacao_dia_publicacao`):** Com ~9.900 e ~10.700 nulos respectivamente, essas colunas têm pouquíssima informação. A hipótese é que a maioria das ocorrências (especialmente incidentes) não gera um relatório público.
    * **Estratégia:** Para o nosso objetivo de analisar as *causas* das ocorrências, essas colunas sobre a *divulgação do relatório* têm pouca utilidade. Removê-las irá simplificar muito nosso `DataFrame` sem grande perda de informação para a nossa análise.

* **Local (`ocorrencia_cidade`, `ocorrencia_uf`):** Possuem uma quantidade muito pequena de nulos (58 e 21).
    * **Estratégia:** O impacto de remover essas poucas linhas do nosso conjunto de dados de mais de 13.000 registros é mínimo. A forma mais simples e limpa de tratar é remover as linhas onde a cidade ou o UF são nulos.

Com base nisso, nosso próximo passo de código será aplicar essa limpeza.

In [4]:
print("--- Aplicando Estratégia para Dados Ausentes ---")

# Vamos continuar trabalhando com o df_ocorrencia_limpo
df_ocorrencia_tratado = df_ocorrencia_limpo.copy()


# 1. Removendo colunas com alta cardinalidade de nulos e pouca relevância para a análise de causas
colunas_para_remover = ['divulgacao_relatorio_numero', 'divulgacao_dia_publicacao']
df_ocorrencia_tratado.drop(columns=colunas_para_remover, inplace=True)

print(f"Colunas removidas: {colunas_para_remover}")


# 2. Removendo linhas onde a cidade ou o UF são nulos (impacto mínimo)
linhas_antes = df_ocorrencia_tratado.shape[0]
df_ocorrencia_tratado.dropna(subset=['ocorrencia_cidade', 'ocorrencia_uf'], inplace=True)
linhas_depois = df_ocorrencia_tratado.shape[0]

print(f"\nLinhas removidas por falta de cidade/UF: {linhas_antes - linhas_depois}")


# --- Resultados ---
print("\nInformações do DataFrame após o tratamento de nulos:")
df_ocorrencia_tratado.info()

--- Aplicando Estratégia para Dados Ausentes ---
Colunas removidas: ['divulgacao_relatorio_numero', 'divulgacao_dia_publicacao']

Linhas removidas por falta de cidade/UF: 0

Informações do DataFrame após o tratamento de nulos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13185 entries, 0 to 13184
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   codigo_ocorrencia               13185 non-null  int64 
 1   ocorrencia_classificacao        13185 non-null  object
 2   ocorrencia_latitude             10425 non-null  object
 3   ocorrencia_longitude            10425 non-null  object
 4   ocorrencia_cidade               13185 non-null  object
 5   ocorrencia_uf                   13185 non-null  object
 6   ocorrencia_pais                 13185 non-null  object
 7   ocorrencia_aerodromo            13185 non-null  object
 8   ocorrencia_dia                  13185 non-null  object
 9  

----------------------------------

## 2. Limpeza e Preparação do `df_aeronave`

Com o nosso `DataFrame` de ocorrências devidamente tratado, vamos agora preparar os dados das aeronaves envolvidas. O processo será semelhante:

1.  **Carregar os Dados:** Carregar o arquivo `aeronave.csv`.
2.  **Padronizar a Chave:** Renomear a coluna `codigo_ocorrencia2` para `codigo_ocorrencia` para permitir a futura junção com o `df_ocorrencia_final`.
3.  **Limpar Valores Sentinela:** Substituir marcadores como `***` e `NULL` por um nulo padrão (`pd.NA`).
4.  **Analisar a Estrutura Final:** Inspecionar os tipos de dados e a nova contagem de valores ausentes.

In [5]:
# --- AERONAVE ---
print("\n\n" + "="*40)
print("INSPECIONANDO O ARQUIVO: aeronave.csv")
print("="*40)
df_aeronave = pd.read_csv('data/aeronave.csv', sep=';', encoding='latin1')
print("\nFormato do DataFrame (linhas, colunas):", df_aeronave.shape)
print("\nInformações das Colunas e Tipos de Dados:")
df_aeronave.info()
print("\nVisualizando as 5 primeiras linhas:")
display(df_aeronave.head())



INSPECIONANDO O ARQUIVO: aeronave.csv

Formato do DataFrame (linhas, colunas): (13301, 23)

Informações das Colunas e Tipos de Dados:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13301 entries, 0 to 13300
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   codigo_ocorrencia2           13301 non-null  int64  
 1   aeronave_matricula           13300 non-null  object 
 2   aeronave_operador_categoria  13292 non-null  object 
 3   aeronave_tipo_veiculo        13300 non-null  object 
 4   aeronave_fabricante          13296 non-null  object 
 5   aeronave_modelo              13291 non-null  object 
 6   aeronave_tipo_icao           13290 non-null  object 
 7   aeronave_motor_tipo          12828 non-null  object 
 8   aeronave_motor_quantidade    13300 non-null  object 
 9   aeronave_pmd                 13300 non-null  float64
 10  aeronave_pmd_categoria       13300 non-null  float64
 

,codigo_ocorrencia2,aeronave_matricula,aeronave_operador_categoria,aeronave_tipo_veiculo,aeronave_fabricante,aeronave_modelo,aeronave_tipo_icao,aeronave_motor_tipo,aeronave_motor_quantidade,aeronave_pmd,...,aeronave_pais_fabricante,aeronave_pais_registro,aeronave_registro_categoria,aeronave_registro_segmento,aeronave_voo_origem,aeronave_voo_destino,aeronave_fase_operacao,aeronave_tipo_operacao,aeronave_nivel_dano,aeronave_fatalidades_total
0,87125,PSAEX,***,AVIÃO,EMBRAER,ERJ 190-400,E295,NaN,***,61200.0,...,BRASIL,BRASIL,AVIÃO,***,VIRACOPOS,ORLANDO BEZERRA DE MENEZES,DESCIDA,REGULAR,NENHUM,0.0
1,87124,PRYXD,***,AVIÃO,ATR - GIE AVIONS DE TRANSPORT RÉGIONAL,ATR-72-212A (600),AT76,***,***,23000.0,...,BRASIL,BRASIL,AVIÃO,***,TENENTE-CORONEL AVIADOR CÉSAR BOMBONATO,TANCREDO NEVES,SUBIDA,REGULAR,NENHUM,0.0
2,87123,PRXBT,***,***,AIRBUS S.A.S.,A320-271N,A20N,NaN,BIMOTOR,79000.0,...,BRASIL,BRASIL,***,***,GOVERNADOR ANDRÉ FRANCO MONTORO,PINTO MARTINS,DECOLAGEM,REGULAR,NENHUM,0.0
3,87122,PRAKJ,***,AVIÃO,ATR - GIE AVIONS DE TRANSPORT RÉGIONAL,ATR-72-212A (600),AT76,TURBOÉLICE,BIMOTOR,22000.0,...,BRASIL,BRASIL,AVIÃO,***,SANTA MARIA,VIRACOPOS,DECOLAGEM,REGULAR,NENHUM,0.0
4,87121,PRMHK,***,AVIÃO,AIRBUS,A320-214,A320,JATO,BIMOTOR,77000.0,...,BRASIL,BRASIL,AVIÃO,***,PROFESSOR ERIBERTO MANOEL REINO,GOVERNADOR ANDRÉ FRANCO MONTORO,DECOLAGEM,REGULAR,LEVE,0.0


In [6]:
print("--- Iniciando processo de limpeza do DataFrame de Aeronaves ---")

# 1. Padronizando a chave de junção
df_aeronave_final = df_aeronave.rename(columns={'codigo_ocorrencia2': 'codigo_ocorrencia'})
print("Etapa 1/3: Chave de junção padronizada para 'codigo_ocorrencia'.")

# 2. Limpando valores sentinela
for col in df_aeronave_final.select_dtypes(include=['object']).columns:
    df_aeronave_final[col] = df_aeronave_final[col].str.strip()
    df_aeronave_final[col] = df_aeronave_final[col].replace(['***', '****', '*****', 'NULL'], pd.NA, regex=False)
print("Etapa 2/3: Valores sentinela substituídos por nulos.")


# --- RESULTADO FINAL DA LIMPEZA DE 'AERONAVE' ---
print("\n--- Limpeza do df_aeronave concluída! ---")
print("\nInformações do DataFrame 'df_aeronave_final':")
df_aeronave_final.info()

print("\nContagem de valores nulos no df_aeronave_final:")
display(df_aeronave_final.isnull().sum())

--- Iniciando processo de limpeza do DataFrame de Aeronaves ---
Etapa 1/3: Chave de junção padronizada para 'codigo_ocorrencia'.
Etapa 2/3: Valores sentinela substituídos por nulos.

--- Limpeza do df_aeronave concluída! ---

Informações do DataFrame 'df_aeronave_final':
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13301 entries, 0 to 13300
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   codigo_ocorrencia            13301 non-null  int64  
 1   aeronave_matricula           13275 non-null  object 
 2   aeronave_operador_categoria  2670 non-null   object 
 3   aeronave_tipo_veiculo        12757 non-null  object 
 4   aeronave_fabricante          13205 non-null  object 
 5   aeronave_modelo              12949 non-null  object 
 6   aeronave_tipo_icao           12767 non-null  object 
 7   aeronave_motor_tipo          12034 non-null  object 
 8   aeronave_motor_quantidade    12734

codigo_ocorrencia                  0
aeronave_matricula                26
aeronave_operador_categoria    10631
aeronave_tipo_veiculo            544
aeronave_fabricante               96
aeronave_modelo                  352
aeronave_tipo_icao               534
aeronave_motor_tipo             1267
aeronave_motor_quantidade        567
aeronave_pmd                       1
aeronave_pmd_categoria             1
aeronave_assentos               1201
aeronave_ano_fabricacao          937
aeronave_pais_fabricante           1
aeronave_pais_registro             1
aeronave_registro_categoria      544
aeronave_registro_segmento      3438
aeronave_voo_origem                6
aeronave_voo_destino               4
aeronave_fase_operacao            30
aeronave_tipo_operacao           989
aeronave_nivel_dano              373
aeronave_fatalidades_total         1
dtype: int64

### 2.1. Análise e Tratamento de Nulos em `df_aeronave`

A limpeza inicial nos deu uma contagem precisa dos dados ausentes. Agora, vamos tratar as colunas mais críticas.

1.  **Investigar `aeronave_registro_segmento`:** Antes de tomar uma decisão, vamos analisar a distribuição de seus valores para entender a informação que ela contém.
2.  **Remover `aeronave_operador_categoria`:** Com mais de 75% de dados ausentes, esta coluna será removida para simplificar o `DataFrame`.

In [7]:
print("--- Análise e Tratamento de Nulos em 'df_aeronave_final' ---")

# 1. Investigando a coluna 'aeronave_registro_segmento'
print("Distribuição de valores para 'aeronave_registro_segmento':")
# O dropna=False nos permite ver a contagem de nulos (NaN) junto com os outros valores
display(df_aeronave_final['aeronave_registro_segmento'].value_counts(dropna=False))


# 2. Removendo a coluna 'aeronave_operador_categoria'
df_aeronave_final.drop(columns=['aeronave_operador_categoria'], inplace=True)
print("\nColuna 'aeronave_operador_categoria' removida.")


# --- RESULTADO ---
print("\nInformações do DataFrame 'df_aeronave_final' após esta etapa:")
df_aeronave_final.info()

--- Análise e Tratamento de Nulos em 'df_aeronave_final' ---
Distribuição de valores para 'aeronave_registro_segmento':


aeronave_registro_segmento
<NA>                      3417
REGULAR                   3046
PARTICULAR                2660
TÁXI AÉREO                1199
INSTRUÇÃO                 1184
AGRÍCOLA                   570
EXPERIMENTAL               568
ADMINISTRAÇÃO DIRETA       284
ESPECIALIZADA              210
NÃO REGULAR                 71
MÚLTIPLA                    47
ADMINISTRAÇÃO INDIRETA      22
NaN                         21
HISTÓRICA                    2
Name: count, dtype: int64


Coluna 'aeronave_operador_categoria' removida.

Informações do DataFrame 'df_aeronave_final' após esta etapa:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13301 entries, 0 to 13300
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   codigo_ocorrencia            13301 non-null  int64  
 1   aeronave_matricula           13275 non-null  object 
 2   aeronave_tipo_veiculo        12757 non-null  object 
 3   aeronave_fabricante          13205 non-null  object 
 4   aeronave_modelo              12949 non-null  object 
 5   aeronave_tipo_icao           12767 non-null  object 
 6   aeronave_motor_tipo          12034 non-null  object 
 7   aeronave_motor_quantidade    12734 non-null  object 
 8   aeronave_pmd                 13300 non-null  float64
 9   aeronave_pmd_categoria       13300 non-null  float64
 10  aeronave_assentos            12100 non-null  float64
 11  aeronave_ano_fabricac

### 2.2. Seleção de Features e Tratamento Final de Nulos em `df_aeronave`

Após a limpeza inicial, o próximo passo é refinar nosso `DataFrame` de aeronaves, selecionando as colunas (features) mais relevantes para a análise e tratando os valores ausentes de forma estratégica.

As seguintes ações serão tomadas:

* **Tratamento de `aeronave_registro_segmento`:**
    * **Observação:** A análise de distribuição desta coluna (executada no passo anterior) mostrou que, embora aproximadamente 25% dos valores sejam nulos, a coluna contém categorias de grande importância para o contexto da ocorrência, como `REGULAR`, `PARTICULAR`, `TÁXI AÉREO` e `INSTRUÇÃO`.
    * **Estratégia:** Remover as linhas com dados nulos resultaria em uma perda significativa de informações. Portanto, a estratégia escolhida é a imputação: os valores ausentes serão preenchidos com a categoria **"NAO INFORMADO"**. Isso preserva os dados e trata a ausência de informação como uma categoria própria, que pode ser relevante para o modelo.

* **Remoção de Colunas Irrelevantes ou Redundantes:** Para simplificar o `DataFrame` e focar nas variáveis mais impactantes, as seguintes colunas serão removidas:
    * `aeronave_matricula`: É um identificador único de cada aeronave, possuindo alta cardinalidade e baixo valor preditivo para uma análise agregada de causas.
    * `aeronave_tipo_icao`: É um código técnico para o modelo da aeronave. A informação principal já está representada de forma mais clara na coluna `aeronave_modelo`, tornando-a redundante.
    * `aeronave_pais_fabricante`: Para o escopo inicial desta análise, focada em fatores operacionais, esta coluna é considerada de menor relevância e será removida para simplificar o modelo.

In [8]:
print("--- Tratamento Final do DataFrame de Aeronaves ---")

# 1. Preenchendo os valores nulos em 'aeronave_registro_segmento'
df_aeronave_final['aeronave_registro_segmento'].fillna('NAO INFORMADO', inplace=True)
print("Etapa 1/3: Valores nulos de 'aeronave_registro_segmento' preenchidos com 'NAO INFORMADO'.")


# 2. Removendo as colunas adicionais selecionadas
colunas_para_remover_aeronave = ['aeronave_matricula', 'aeronave_tipo_icao', 'aeronave_pais_fabricante']
df_aeronave_final.drop(columns=colunas_para_remover_aeronave, inplace=True)
print(f"Etapa 2/3: Colunas removidas: {colunas_para_remover_aeronave}")


# 3. Verificando a distribuição de 'aeronave_registro_segmento' após o tratamento
print("\nDistribuição de valores para 'aeronave_registro_segmento' (após tratamento):")
display(df_aeronave_final['aeronave_registro_segmento'].value_counts(dropna=False))


# --- RESULTADO FINAL DA LIMPEZA DE 'AERONAVE' ---
print("\n--- Limpeza do df_aeronave concluída! ---")
print("\nInformações do DataFrame 'df_aeronave_final' após limpeza completa:")
df_aeronave_final.info()

--- Tratamento Final do DataFrame de Aeronaves ---
Etapa 1/3: Valores nulos de 'aeronave_registro_segmento' preenchidos com 'NAO INFORMADO'.
Etapa 2/3: Colunas removidas: ['aeronave_matricula', 'aeronave_tipo_icao', 'aeronave_pais_fabricante']

Distribuição de valores para 'aeronave_registro_segmento' (após tratamento):


C:\Users\charl\AppData\Local\Temp\ipykernel_3172\859675127.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_aeronave_final['aeronave_registro_segmento'].fillna('NAO INFORMADO', inplace=True)


aeronave_registro_segmento
NAO INFORMADO             3438
REGULAR                   3046
PARTICULAR                2660
TÁXI AÉREO                1199
INSTRUÇÃO                 1184
AGRÍCOLA                   570
EXPERIMENTAL               568
ADMINISTRAÇÃO DIRETA       284
ESPECIALIZADA              210
NÃO REGULAR                 71
MÚLTIPLA                    47
ADMINISTRAÇÃO INDIRETA      22
HISTÓRICA                    2
Name: count, dtype: int64


--- Limpeza do df_aeronave concluída! ---

Informações do DataFrame 'df_aeronave_final' após limpeza completa:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13301 entries, 0 to 13300
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   codigo_ocorrencia            13301 non-null  int64  
 1   aeronave_tipo_veiculo        12757 non-null  object 
 2   aeronave_fabricante          13205 non-null  object 
 3   aeronave_modelo              12949 non-null  object 
 4   aeronave_motor_tipo          12034 non-null  object 
 5   aeronave_motor_quantidade    12734 non-null  object 
 6   aeronave_pmd                 13300 non-null  float64
 7   aeronave_pmd_categoria       13300 non-null  float64
 8   aeronave_assentos            12100 non-null  float64
 9   aeronave_ano_fabricacao      12364 non-null  float64
 10  aeronave_pais_registro       13300 non-null  object 
 11  aeronave_registro_ca

## 3. Limpeza e Preparação do `df_ocorrencia_tipo`

Com os dados de ocorrência e aeronave tratados, o próximo passo é preparar a tabela de tipos de ocorrência. O objetivo é padronizar a chave de junção e simplificar o `DataFrame`, removendo redundâncias e dados ausentes.

1.  **Padronização da Chave:** A coluna `codigo_ocorrencia1` será renomeada para `codigo_ocorrencia` para manter a consistência com os outros `DataFrames`.
2.  **Análise de Redundância:** Será verificado se o conteúdo das colunas `ocorrencia_tipo` e `ocorrencia_tipo_categoria` é idêntico.
3.  **Limpeza de Nulos:** As 5 linhas que contêm valores nulos serão removidas, dado seu número insignificante.

In [6]:
# --- OCORRÊNCIA TIPO ---
print("\n\n" + "="*40)
print("INSPECIONANDO O ARQUIVO: ocorrencia_tipo.csv")
print("="*40)
df_ocorrencia_tipo = pd.read_csv('data/ocorrencia_tipo.csv', sep=';', encoding='latin1')
print("\nFormato do DataFrame (linhas, colunas):", df_ocorrencia_tipo.shape)
print("\nInformações das Colunas e Tipos de Dados:")
df_ocorrencia_tipo.info()
print("\nVisualizando as 5 primeiras linhas:")
display(df_ocorrencia_tipo.head())



INSPECIONANDO O ARQUIVO: ocorrencia_tipo.csv

Formato do DataFrame (linhas, colunas): (13900, 4)

Informações das Colunas e Tipos de Dados:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13900 entries, 0 to 13899
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   codigo_ocorrencia1         13900 non-null  int64 
 1   ocorrencia_tipo            13895 non-null  object
 2   ocorrencia_tipo_categoria  13895 non-null  object
 3   taxonomia_tipo_icao        13895 non-null  object
dtypes: int64(1), object(3)
memory usage: 434.5+ KB

Visualizando as 5 primeiras linhas:


,codigo_ocorrencia1,ocorrencia_tipo,ocorrencia_tipo_categoria,taxonomia_tipo_icao
0,87125,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP
1,87124,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP
2,87123,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP
3,87122,FALHA OU MAU FUNCIONAMENTO DO MOTOR,FALHA OU MAU FUNCIONAMENTO DO MOTOR,SCF-PP
4,87121,COLISÃO COM AVE,COLISÃO COM AVE,BIRD


In [9]:
print("--- Iniciando processo de limpeza do DataFrame de Tipos de Ocorrência ---")

# 1. Carregando o arquivo
df_ocorrencia_tipo = pd.read_csv('data/ocorrencia_tipo.csv', sep=';', encoding='latin1')

# 2. Padronizando a chave de junção
df_ocorrencia_tipo_final = df_ocorrencia_tipo.rename(columns={'codigo_ocorrencia1': 'codigo_ocorrencia'})
print("Etapa 1/4: Chave de junção padronizada para 'codigo_ocorrencia'.")

# 3. Removendo os poucos valores nulos
df_ocorrencia_tipo_final.dropna(inplace=True)
print("Etapa 2/4: Linhas com valores nulos removidas.")

# 4. Verificando se as colunas de tipo são idênticas
sao_identicas_tipo = (df_ocorrencia_tipo_final['ocorrencia_tipo'] == df_ocorrencia_tipo_final['ocorrencia_tipo_categoria']).all()
print(f"Etapa 3/4: As colunas 'ocorrencia_tipo' e 'ocorrencia_tipo_categoria' são idênticas? -> {sao_identicas_tipo}")

# 5. Se forem idênticas, removemos a redundância
if sao_identicas_tipo:
    df_ocorrencia_tipo_final.drop(columns=['ocorrencia_tipo_categoria'], inplace=True)
    print("Etapa 4/4: Coluna 'ocorrencia_tipo_categoria' removida por redundância.")


# --- RESULTADO FINAL DA LIMPEZA ---
print("\n--- Limpeza do df_ocorrencia_tipo concluída! ---")
print("\nInformações do DataFrame 'df_ocorrencia_tipo_final':")
df_ocorrencia_tipo_final.info()

print("\nVisualizando as 5 primeiras linhas do resultado final:")
display(df_ocorrencia_tipo_final.head())

--- Iniciando processo de limpeza do DataFrame de Tipos de Ocorrência ---
Etapa 1/4: Chave de junção padronizada para 'codigo_ocorrencia'.
Etapa 2/4: Linhas com valores nulos removidas.
Etapa 3/4: As colunas 'ocorrencia_tipo' e 'ocorrencia_tipo_categoria' são idênticas? -> False

--- Limpeza do df_ocorrencia_tipo concluída! ---

Informações do DataFrame 'df_ocorrencia_tipo_final':
<class 'pandas.core.frame.DataFrame'>
Index: 13895 entries, 0 to 13899
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   codigo_ocorrencia          13895 non-null  int64 
 1   ocorrencia_tipo            13895 non-null  object
 2   ocorrencia_tipo_categoria  13895 non-null  object
 3   taxonomia_tipo_icao        13895 non-null  object
dtypes: int64(1), object(3)
memory usage: 542.8+ KB

Visualizando as 5 primeiras linhas do resultado final:


,codigo_ocorrencia,ocorrencia_tipo,ocorrencia_tipo_categoria,taxonomia_tipo_icao
0,87125,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP
1,87124,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP
2,87123,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP
3,87122,FALHA OU MAU FUNCIONAMENTO DO MOTOR,FALHA OU MAU FUNCIONAMENTO DO MOTOR,SCF-PP
4,87121,COLISÃO COM AVE,COLISÃO COM AVE,BIRD


### 3.1. Investigando a Divergência entre `ocorrencia_tipo` e `ocorrencia_tipo_categoria`

A nossa verificação retornou `False`, indicando que as colunas `ocorrencia_tipo` e `ocorrencia_tipo_categoria` não são idênticas. Para entender a natureza dessa diferença, o próximo passo é filtrar e exibir apenas as linhas onde os valores destas duas colunas divergem.

Esta análise nos permitirá entender a relação entre elas e decidir se:
* Uma coluna é simplesmente uma versão mais detalhada da outra.
* Ambas as colunas contêm informações únicas e devem ser mantidas.
* A diferença é apenas um erro de preenchimento em poucos casos.

In [10]:
print("--- Exibindo as linhas com valores divergentes entre 'ocorrencia_tipo' e 'ocorrencia_tipo_categoria' ---")

# Criando um filtro booleano para encontrar as linhas onde os valores são diferentes
filtro_divergencia = df_ocorrencia_tipo_final['ocorrencia_tipo'] != df_ocorrencia_tipo_final['ocorrencia_tipo_categoria']

# Aplicando o filtro para criar um novo DataFrame apenas com as divergências
df_divergencias = df_ocorrencia_tipo_final[filtro_divergencia]

print(f"\nForam encontradas {df_divergencias.shape[0]} linhas com valores divergentes.")

# Exibindo todas as linhas divergentes
# Usamos display() para uma melhor formatação no notebook
display(df_divergencias)

--- Exibindo as linhas com valores divergentes entre 'ocorrencia_tipo' e 'ocorrencia_tipo_categoria' ---

Foram encontradas 4896 linhas com valores divergentes.


,codigo_ocorrencia,ocorrencia_tipo,ocorrencia_tipo_categoria,taxonomia_tipo_icao
16,87108,CONTATO ANORMAL COM A PISTAA,CONTATO ANORMAL COM A PISTA |,ARC
17,87107,CONTATO ANORMAL COM A PISTAA,CONTATO ANORMAL COM A PISTA |,ARC
119,86993,CONTATO ANORMAL COM A PISTAA,CONTATO ANORMAL COM A PISTA |,ARC
183,86913,CONTATO ANORMAL COM A PISTAA,CONTATO ANORMAL COM A PISTA |,ARC
189,86907,CONTATO ANORMAL COM A PISTAA,CONTATO ANORMAL COM A PISTA |,ARC
...,...,...,...,...
13889,28475,POUSO SEM TREM,CONTATO ANORMAL COM A PISTA | POUSO SEM TREM,ARC
13891,28437,FALHA DO MOTOR EM VOO,FALHA OU MAU FUNCIONAMENTO DO MOTOR | FALHA DO...,SCF-PP
13892,28395,FALHA ESTRUTURAL,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...,SCF-NP
13895,28377,FALHA DO MOTOR EM VOO,FALHA OU MAU FUNCIONAMENTO DO MOTOR | FALHA DO...,SCF-PP


### 3.2. Limpeza e Padronização das Colunas de Tipo de Ocorrência

A investigação das divergências nos mostrou que `ocorrencia_tipo_categoria` é uma classificação mais ampla que engloba o `ocorrencia_tipo`. Ambas são úteis, portanto, serão mantidas.

Nesta etapa, vamos realizar uma limpeza focada para corrigir os problemas de formatação e digitação que encontramos:

1.  **Correção de Erros:** Corrigir erros de digitação específicos, como "PISTAA", na coluna `ocorrencia_tipo`.
2.  **Limpeza de Formatação:** Remover caracteres extras, como a barra `|` e espaços em branco, do final dos valores na coluna `ocorrencia_tipo_categoria`.

In [11]:
print("--- Limpando e padronizando as colunas de tipo de ocorrência ---")

# 1. Corrigindo o erro de digitação em 'ocorrencia_tipo'
df_ocorrencia_tipo_final['ocorrencia_tipo'] = df_ocorrencia_tipo_final['ocorrencia_tipo'].replace({
    'CONTATO ANORMAL COM A PISTAA': 'CONTATO ANORMAL COM A PISTA'
})
print("Etapa 1/2: Erro de digitação 'PISTAA' corrigido.")

# 2. Removendo espaços e barras '|' do início e do fim da coluna 'ocorrencia_tipo_categoria'
df_ocorrencia_tipo_final['ocorrencia_tipo_categoria'] = df_ocorrencia_tipo_final['ocorrencia_tipo_categoria'].str.strip(' |')
print("Etapa 2/2: Formatação da coluna 'ocorrencia_tipo_categoria' corrigida.")


# --- Verificação Pós-Limpeza ---
print("\n--- Verificando as divergências novamente após a limpeza ---")
filtro_divergencia_limpo = df_ocorrencia_tipo_final['ocorrencia_tipo'] != df_ocorrencia_tipo_final['ocorrencia_tipo_categoria']
df_divergencias_limpo = df_ocorrencia_tipo_final[filtro_divergencia_limpo]

print(f"\nNúmero de linhas divergentes após a limpeza: {df_divergencias_limpo.shape[0]}")


# --- RESULTADO FINAL DA LIMPEZA ---
print("\n--- Limpeza do df_ocorrencia_tipo concluída! ---")
print("\nInformações do DataFrame 'df_ocorrencia_tipo_final':")
df_ocorrencia_tipo_final.info()

--- Limpando e padronizando as colunas de tipo de ocorrência ---
Etapa 1/2: Erro de digitação 'PISTAA' corrigido.
Etapa 2/2: Formatação da coluna 'ocorrencia_tipo_categoria' corrigida.

--- Verificando as divergências novamente após a limpeza ---

Número de linhas divergentes após a limpeza: 4761

--- Limpeza do df_ocorrencia_tipo concluída! ---

Informações do DataFrame 'df_ocorrencia_tipo_final':
<class 'pandas.core.frame.DataFrame'>
Index: 13895 entries, 0 to 13899
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   codigo_ocorrencia          13895 non-null  int64 
 1   ocorrencia_tipo            13895 non-null  object
 2   ocorrencia_tipo_categoria  13895 non-null  object
 3   taxonomia_tipo_icao        13895 non-null  object
dtypes: int64(1), object(3)
memory usage: 542.8+ KB


## 4. Limpeza e Preparação do `df_fator_contribuinte`

Agora que os `DataFrames` de ocorrência e tipo de ocorrência estão prontos, vamos focar no arquivo que detalha os fatores que contribuíram para as ocorrências.

A primeira etapa, como sempre, é a inspeção inicial da estrutura e do conteúdo do arquivo.

In [12]:
# --- FATOR CONTRIBUINTE ---
print("\n\n" + "="*40)
print("INSPECIONANDO O ARQUIVO: fator_contribuinte.csv")
print("="*40)
df_fator_contribuinte = pd.read_csv('data/fator_contribuinte.csv', sep=';', encoding='latin1')
print("\nFormato do DataFrame (linhas, colunas):", df_fator_contribuinte.shape)
print("\nInformações das Colunas e Tipos de Dados:")
df_fator_contribuinte.info()
print("\nVisualizando as 5 primeiras linhas:")
display(df_fator_contribuinte.head())



INSPECIONANDO O ARQUIVO: fator_contribuinte.csv

Formato do DataFrame (linhas, colunas): (8613, 5)

Informações das Colunas e Tipos de Dados:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8613 entries, 0 to 8612
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   codigo_ocorrencia3   8613 non-null   int64 
 1   fator_nome           8613 non-null   object
 2   fator_aspecto        8613 non-null   object
 3   fator_condicionante  8613 non-null   object
 4   fator_area           8613 non-null   object
dtypes: int64(1), object(4)
memory usage: 336.6+ KB

Visualizando as 5 primeiras linhas:


,codigo_ocorrencia3,fator_nome,fator_aspecto,fator_condicionante,fator_area
0,85239,APLICAÇÃO DE COMANDOS,DESEMPENHO DO SER HUMANO,OPERAÇÃO DA AERONAVE,FATOR OPERACIONAL
1,85239,ATITUDE,ASPECTO PSICOLÓGICO,INDIVIDUAL,FATOR HUMANO
2,85239,CONDIÇÕES METEOROLÓGICAS ADVERSAS,ELEMENTOS RELACIONADOS AO AMBIENTE OPERACIONAL,***,FATOR OPERACIONAL
3,85239,PERCEPÇÃO,ASPECTO PSICOLÓGICO,INDIVIDUAL,FATOR HUMANO
4,85239,PROCESSO DECISÓRIO,ASPECTO PSICOLÓGICO,INDIVIDUAL,FATOR HUMANO


### 4.1 Limpeza e Preparação do `df_fator_contribuinte`

Este `DataFrame` é crucial para a análise, pois detalha os fatores que contribuíram para as ocorrências. A preparação seguirá nosso processo padrão:

1.  **Padronização da Chave:** A coluna `codigo_ocorrencia3` será renomeada para `codigo_ocorrencia`.
2.  **Limpeza de Valores Sentinela:** O valor `***` presente na coluna `fator_condicionante` será substituído por um nulo padrão (`pd.NA`) para garantir a consistência dos dados.

In [13]:
print("--- Iniciando processo de limpeza do DataFrame de Fatores Contribuintes ---")

# 1. Carregando o arquivo
df_fator_contribuinte = pd.read_csv('data/fator_contribuinte.csv', sep=';', encoding='latin1')

# 2. Padronizando a chave de junção
df_fator_contribuinte_final = df_fator_contribuinte.rename(columns={'codigo_ocorrencia3': 'codigo_ocorrencia'})
print("Etapa 1/2: Chave de junção padronizada para 'codigo_ocorrencia'.")

# 3. Limpando valores sentinela
df_fator_contribuinte_final.replace('***', pd.NA, inplace=True)
print("Etapa 2/2: Valores sentinela ('***') substituídos por nulos.")


# --- RESULTADO FINAL DA LIMPEZA ---
print("\n--- Limpeza do df_fator_contribuinte concluída! ---")
print("\nInformações do DataFrame 'df_fator_contribuinte_final':")
df_fator_contribuinte_final.info()

print("\nContagem de valores nulos após a limpeza:")
display(df_fator_contribuinte_final.isnull().sum())

--- Iniciando processo de limpeza do DataFrame de Fatores Contribuintes ---
Etapa 1/2: Chave de junção padronizada para 'codigo_ocorrencia'.
Etapa 2/2: Valores sentinela ('***') substituídos por nulos.

--- Limpeza do df_fator_contribuinte concluída! ---

Informações do DataFrame 'df_fator_contribuinte_final':
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8613 entries, 0 to 8612
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   codigo_ocorrencia    8613 non-null   int64 
 1   fator_nome           8613 non-null   object
 2   fator_aspecto        8613 non-null   object
 3   fator_condicionante  7705 non-null   object
 4   fator_area           8613 non-null   object
dtypes: int64(1), object(4)
memory usage: 336.6+ KB

Contagem de valores nulos após a limpeza:


codigo_ocorrencia        0
fator_nome               0
fator_aspecto            0
fator_condicionante    908
fator_area               0
dtype: int64

### 4.2. Tratamento de Nulos em `df_fator_contribuinte`

A limpeza inicial revelou quase 1.000 valores ausentes na coluna `fator_condicionante`. Para finalizar a preparação deste `DataFrame`, vamos imputar esses valores.

* **Estratégia:** Preencher os valores nulos com a categoria **"NAO INFORMADO"**. Isso mantém a integridade dos dados e garante que todas as linhas possam ser utilizadas em análises futuras.

In [14]:
print("--- Tratamento Final do DataFrame de Fatores Contribuintes ---")

# 1. Preenchendo os valores nulos em 'fator_condicionante'
df_fator_contribuinte_final['fator_condicionante'].fillna('NAO INFORMADO', inplace=True)
print("Valores nulos de 'fator_condicionante' preenchidos com 'NAO INFORMADO'.")

# --- RESULTADO FINAL DA LIMPEZA ---
print("\n--- Limpeza do df_fator_contribuinte concluída! ---")
print("\nContagem de valores nulos após o tratamento final:")
display(df_fator_contribuinte_final.isnull().sum())

--- Tratamento Final do DataFrame de Fatores Contribuintes ---
Valores nulos de 'fator_condicionante' preenchidos com 'NAO INFORMADO'.

--- Limpeza do df_fator_contribuinte concluída! ---

Contagem de valores nulos após o tratamento final:


C:\Users\charl\AppData\Local\Temp\ipykernel_3172\1235658836.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_fator_contribuinte_final['fator_condicionante'].fillna('NAO INFORMADO', inplace=True)


codigo_ocorrencia      0
fator_nome             0
fator_aspecto          0
fator_condicionante    0
fator_area             0
dtype: int64

## 5. Limpeza e Preparação do `df_recomendacao`

Chegamos ao último arquivo de dados. Esta tabela contém as recomendações de segurança emitidas após as investigações. Vamos aplicar nosso processo padrão de inspeção e limpeza.

In [15]:
# --- RECOMENDAÇÃO ---
print("\n\n" + "="*40)
print("INSPECIONANDO O ARQUIVO: recomendacao.csv")
print("="*40)
df_recomendacao = pd.read_csv('data/recomendacao.csv', sep=';', encoding='latin1')
print("\nFormato do DataFrame (linhas, colunas):", df_recomendacao.shape)
print("\nInformações das Colunas e Tipos de Dados:")
df_recomendacao.info()
print("\nVisualizando as 5 primeiras linhas:")
display(df_recomendacao.head())



INSPECIONANDO O ARQUIVO: recomendacao.csv

Formato do DataFrame (linhas, colunas): (3388, 9)

Informações das Colunas e Tipos de Dados:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3388 entries, 0 to 3387
Data columns (total 9 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   codigo_ocorrencia4               3387 non-null   object
 1   recomendacao_numero              2890 non-null   object
 2   recomendacao_dia_assinatura      2890 non-null   object
 3   recomendacao_dia_encaminhamento  2890 non-null   object
 4   recomendacao_dia_feedback        2217 non-null   object
 5   recomendacao_conteudo            2889 non-null   object
 6   recomendacao_status              2889 non-null   object
 7   recomendacao_destinatario_sigla  2889 non-null   object
 8   recomendacao_destinatario        2889 non-null   object
dtypes: object(9)
memory usage: 238.3+ KB

Visualizando as 5 primeiras linhas:


,codigo_ocorrencia4,recomendacao_numero,recomendacao_dia_assinatura,recomendacao_dia_encaminhamento,recomendacao_dia_feedback,recomendacao_conteudo,recomendacao_status,recomendacao_destinatario_sigla,recomendacao_destinatario
0,84857,A-092/CENIPA/2024 - 01,2025-04-22,2025-04-23,NaN,Divulgar os ensinamentos colhidos na presente ...,AGUARDANDO RESPOSTA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
1,83193,A-014/CENIPA/2024 - 01,2025-02-17,2025-02-19,NaN,Divulgar os ensinamentos colhidos na presente ...,AGUARDANDO RESPOSTA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
2,83193,A-014/CENIPA/2024 - 02,2025-02-17,2025-02-19,2025-03-07,Divulgar os ensinamentos colhidos na presente ...,ADOTADA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
3,82458,A-146/CENIPA/2023 - 01,2025-03-13,2025-04-08,2025-05-09,Divulgar os ensinamentos colhidos na presente ...,ADOTADA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL
4,82458,A-146/CENIPA/2023 - 02,2025-03-13,2025-04-08,NaN,Avaliar a pertinência de revisar os requisitos...,AGUARDANDO RESPOSTA,ANAC,AGÊNCIA NACIONAL DE AVIAÇÃO CIVIL


### 5.1 Decisão de Escopo: Exclusão do `df_recomendacao`

Após a inspeção inicial do arquivo `recomendacao.csv`, foi identificado que seus dados se referem às ações e recomendações de segurança geradas *após* a investigação das ocorrências.

Para o nosso objetivo principal, que é analisar as **causas** e construir um modelo preditivo baseado nas **características do evento em si**, esta tabela representa dados "pós-evento". Utilizá-la como uma *feature* preditiva constituiria um vazamento de dados (*data leakage*), pois o modelo teria acesso a informações do "futuro" para prever o evento.

Portanto, para manter o foco e a integridade da análise, foi tomada a decisão estratégica de **excluir esta tabela do escopo de trabalho deste projeto.**

# Fase 2: Unificação dos Dados e Análise Exploratória (EDA)

Com todos os `DataFrames` individuais limpos e preparados, o próximo passo é unificá-los em uma única tabela. Isso nos dará uma visão 360º de cada ocorrência, combinando informações do evento, da aeronave, do tipo de ocorrência e de seus fatores contribuintes.

Vamos realizar as junções (`merges`) usando nossa chave padronizada, `codigo_ocorrencia`.

In [16]:
print("--- Unificando todos os DataFrames limpos ---")

# Vamos começar com o df_ocorrencia_tratado, que é nossa tabela principal
df_master = df_ocorrencia_tratado.copy()

# Lista dos dataframes para juntar
dataframes_para_juntar = [
    df_ocorrencia_tipo_final,
    df_aeronave_final,
    df_fator_contribuinte_final
]

# Loop para fazer o merge de cada dataframe com o df_master
# Usamos how='left' para garantir que manteremos todas as ocorrências originais,
# mesmo que alguma não tenha correspondência nas outras tabelas.
for df in dataframes_para_juntar:
    df_master = pd.merge(df_master, df, on='codigo_ocorrencia', how='left')

print("Junção de todos os DataFrames concluída!")


# --- RESULTADO FINAL DA JUNÇÃO ---
print("\n--- Inspeção do DataFrame Mestre Unificado ---")
print("\nInformações do DataFrame 'df_master':")
df_master.info()

print("\nVisualizando as 5 primeiras linhas do resultado final:")
display(df_master.head())

--- Unificando todos os DataFrames limpos ---
Junção de todos os DataFrames concluída!

--- Inspeção do DataFrame Mestre Unificado ---

Informações do DataFrame 'df_master':
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21486 entries, 0 to 21485
Data columns (total 41 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   codigo_ocorrencia               21486 non-null  int64  
 1   ocorrencia_classificacao        21486 non-null  object 
 2   ocorrencia_latitude             18682 non-null  object 
 3   ocorrencia_longitude            18682 non-null  object 
 4   ocorrencia_cidade               21486 non-null  object 
 5   ocorrencia_uf                   21486 non-null  object 
 6   ocorrencia_pais                 21486 non-null  object 
 7   ocorrencia_aerodromo            21486 non-null  object 
 8   ocorrencia_dia                  21486 non-null  object 
 9   ocorrencia_hora                 21484 non

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,ocorrencia_pais,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,...,aeronave_voo_origem,aeronave_voo_destino,aeronave_fase_operacao,aeronave_tipo_operacao,aeronave_nivel_dano,aeronave_fatalidades_total,fator_nome,fator_aspecto,fator_condicionante,fator_area
0,87125,INCIDENTE,-7.219166666666,-39.26944444444,JUAZEIRO DO NORTE,CE,BRASIL,FAER,11/05/2025,04:20:00,...,VIRACOPOS,ORLANDO BEZERRA DE MENEZES,DESCIDA,REGULAR,NENHUM,0.0,NaN,NaN,NaN,NaN
1,87124,INCIDENTE,-18.88361111111,-48.22527777777,UBERLÂNDIA,MG,BRASIL,SBUL,08/05/2025,14:00:00,...,TENENTE-CORONEL AVIADOR CÉSAR BOMBONATO,TANCREDO NEVES,SUBIDA,REGULAR,NENHUM,0.0,NaN,NaN,NaN,NaN
2,87123,INCIDENTE,-23.43555555555,-46.47305555555,GUARULHOS,SP,BRASIL,SBGR,09/05/2025,18:45:00,...,GOVERNADOR ANDRÉ FRANCO MONTORO,PINTO MARTINS,DECOLAGEM,REGULAR,NENHUM,0.0,NaN,NaN,NaN,NaN
3,87122,INCIDENTE,-29.71083333333,-53.69222222222,SANTA MARIA,RS,BRASIL,SBSM,04/05/2025,14:45:00,...,SANTA MARIA,VIRACOPOS,DECOLAGEM,REGULAR,NENHUM,0.0,NaN,NaN,NaN,NaN
4,87121,INCIDENTE,-20.81722222222,-49.40694444444,SÃO JOSÉ DO RIO PRETO,SP,BRASIL,SBSR,10/05/2025,10:00:00,...,PROFESSOR ERIBERTO MANOEL REINO,GOVERNADOR ANDRÉ FRANCO MONTORO,DECOLAGEM,REGULAR,LEVE,0.0,NaN,NaN,NaN,NaN


# Fase 3: Salvando o DataFrame Mestre

Com todos os DataFrames limpos, preparados e unificados, a etapa de preparação dos dados está concluída.

Vamos salvar o nosso `df_master` em um novo arquivo CSV. Este arquivo será a fonte de dados para o nosso próximo notebook, focado exclusivamente na Análise Exploratória dos Dados (EDA).

In [18]:
print("--- Salvando o DataFrame Mestre em um novo arquivo CSV ---")

# Definindo o nome do arquivo de saída
output_path = 'data/cenipa_master.csv'

# Salvando o DataFrame
# index=False evita que o índice do pandas seja salvo como uma coluna no CSV
df_master.to_csv(output_path, index=False, sep=';', encoding='utf-8-sig')

print(f"\nDataFrame unificado salvo com sucesso em: {output_path}")
print(f"Total de {df_master.shape[0]} linhas e {df_master.shape[1]} colunas.")

--- Salvando o DataFrame Mestre em um novo arquivo CSV ---

DataFrame unificado salvo com sucesso em: data/cenipa_master.csv
Total de 21486 linhas e 41 colunas.
